## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from gensim.models import LsiModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


## Read cleaned data files

### Read cleaned jobs listings file and save in dataframe

In [2]:
cleaned_jobs_listings_df = pd.read_csv("cleaned_jobs_listings.csv")

### Check columns info in jobs listings dataframe

In [3]:
cleaned_jobs_listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3690 entries, 0 to 3689
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   job_title                3690 non-null   object 
 1   salary_estimate          3690 non-null   object 
 2   job_description          3690 non-null   object 
 3   rating                   3690 non-null   float64
 4   company_name             3690 non-null   object 
 5   location                 3690 non-null   object 
 6   headquarters             3690 non-null   object 
 7   size                     3690 non-null   object 
 8   founded                  3690 non-null   int64  
 9   type_of_ownership        3690 non-null   object 
 10  industry                 3690 non-null   object 
 11  sector                   3690 non-null   object 
 12  revenue                  3690 non-null   object 
 13  competitors              3690 non-null   object 
 14  easy_apply              

## Data Preprocessing

### Define stopwords to append to default stopwords

In [4]:
# append special stopwords in jobs listings to default and common stopwords
new_stopwords = STOPWORDS.union(set(['job', 'skill', 'experience', 'team', 'data', 'use', 'like', 'business', 
                                'work', 'ability', 'let', 'need', 'new', 'user', 'opportunity', 
                                'candidate', 'provide', 'company', 'one', 'used', 'need', 'see', 'make', 
                                'follow', 'going', 'will', 'want', 'well', 'find', 'give', 'change', 'look', 
                                'first', 'using','know', 'science', 'think', 'year', 'years', 'looking',
                                'including', 'working', 'scientist', 'employee', 'skills', 'scientist',
                                'knowledge', 'position', 'technology', 'computer']))

### Method to preprocess data for jobs listings

In [5]:
# lemmatize = words change to noun
# stemmed = words reduced to root form
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='n')

# Convert a document into a list of tokens.
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in new_stopwords and len(token) > 3:
            result.append(lemmatize(token))
    return result

In [6]:
#preprocess the jobs listings
preprocessed_jobs_listings = cleaned_jobs_listings_df['cleaned_job_description'].fillna('').astype(str).map(preprocess)

In [7]:
# implements the concept of a Dictionary – a mapping between words and their integer ids.
jobs_listings_dictionary = corpora.Dictionary(preprocessed_jobs_listings)

In [8]:
# Convert into the bag-of-words (BoW) format
jobs_listings_bow_corpus = [jobs_listings_dictionary.doc2bow(text) for text in preprocessed_jobs_listings]

In [9]:
# Convert into TF-IDF format
jobs_listings_tfidf = models.TfidfModel(jobs_listings_bow_corpus)
jobs_listings_tfidf_corpus = jobs_listings_tfidf[jobs_listings_bow_corpus]

## Common method to find top 10 dominant topics in the jobs description

In [10]:
def get_dominant_topics(model, corpus):
    dominant_topics_df = pd.DataFrame()
    for i, row in enumerate(model[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0: # => dominant topic
                wp = model.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                dominant_topics_df = dominant_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    dominant_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    dominant_topics_df["Text"] = cleaned_jobs_listings_df['cleaned_job_description']
    dominant_topics_df = dominant_topics_df.sort_values(by=['Perc_Contribution'], ascending=False)
    return dominant_topics_df.head(10)

## LSA Model

### Result using LSA model + Bag of words

In [11]:
jobs_listings_bow_lsamodel = LsiModel(jobs_listings_bow_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [12]:
jobs_listings_bow_lsamodel.print_topics(num_words=7)

[(0,
  '0.184*"development" + 0.175*"analysis" + 0.153*"product" + 0.147*"analytics" + 0.143*"information" + 0.142*"solution" + 0.136*"project"'),
 (1,
  '-0.276*"learning" + -0.226*"analytics" + -0.225*"machine" + 0.209*"information" + -0.186*"model" + 0.180*"laboratory" + 0.163*"research"'),
 (2,
  '0.396*"learning" + 0.318*"machine" + -0.183*"process" + 0.173*"research" + -0.165*"management" + -0.152*"development" + 0.147*"application"'),
 (3,
  '-0.420*"research" + 0.278*"information" + 0.184*"application" + -0.184*"product" + -0.178*"laboratory" + -0.173*"clinical" + 0.147*"solution"'),
 (4,
  '0.408*"development" + -0.292*"analysis" + -0.264*"analytics" + 0.229*"product" + -0.176*"statistical" + 0.147*"cell" + 0.142*"design"'),
 (5,
  '0.329*"research" + -0.313*"product" + -0.203*"status" + -0.183*"people" + 0.168*"development" + -0.160*"employment" + 0.151*"learning"'),
 (6,
  '0.351*"laboratory" + -0.251*"research" + -0.212*"analysis" + 0.211*"service" + -0.207*"product" + 0.20

In [13]:
get_dominant_topics(jobs_listings_bow_lsamodel, jobs_listings_bow_corpus)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
591,0.0,37.2716,"development, analysis, product, analytics, inf...",job descriptionoperation analytics manager tr...
1889,0.0,34.5236,"development, analysis, product, analytics, inf...",job descriptionjob title data analystsalary 18...
891,0.0,30.6456,"development, analysis, product, analytics, inf...",at west monroe our people are our businesswe p...
1485,0.0,30.0066,"development, analysis, product, analytics, inf...",descriptionthe lead of cybersecurity data prot...
788,0.0,29.1049,"development, analysis, product, analytics, inf...",address111 w monroe 115 s lasallejob family g...
97,0.0,26.6167,"development, analysis, product, analytics, inf...",search by keywordsearch by locationclearsend m...
1199,6.0,26.5056,"laboratory, research, analysis, service, produ...",work shift eveningwork week m fjob summary082...
874,0.0,26.3517,"development, analysis, product, analytics, inf...",please make sure to read the job posting in it...
770,0.0,25.8184,"development, analysis, product, analytics, inf...",address111 w monroe 115 s lasallejob family g...
1185,6.0,25.5784,"laboratory, research, analysis, service, produ...",work shift daywork week this position will wor...


### Result using LSA model + TF-IDF

In [14]:
jobs_listings_tfidf_lsamodel = LsiModel(jobs_listings_tfidf_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [15]:
jobs_listings_tfidf_lsamodel.print_topics(num_words=7)

[(0,
  '0.100*"learning" + 0.097*"analytics" + 0.095*"machine" + 0.089*"product" + 0.087*"model" + 0.079*"customer" + 0.079*"research"'),
 (1,
  '0.274*"laboratory" + 0.182*"clinical" + 0.164*"cell" + -0.133*"machine" + -0.132*"learning" + 0.120*"assay" + 0.114*"scientific"'),
 (2,
  '-0.186*"analyze" + -0.180*"locate" + -0.172*"printout" + -0.168*"databasesdata" + -0.154*"database" + -0.153*"setsfilter" + -0.153*"qualityacquire"'),
 (3,
  '-0.179*"university" + 0.174*"cell" + -0.141*"clery" + -0.141*"security" + -0.126*"investigation" + 0.116*"clinical" + 0.114*"assay"'),
 (4,
  '-0.727*"tutor" + -0.487*"tutoring" + -0.178*"student" + -0.131*"choose" + -0.123*"session" + -0.088*"schedule" + -0.081*"tutoringonline"'),
 (5,
  '-0.149*"pipeline" + -0.139*"cloud" + -0.125*"spark" + -0.123*"bull" + 0.122*"machine" + -0.118*"laboratory" + 0.112*"learning"'),
 (6,
  '-0.226*"cell" + -0.175*"machine" + -0.172*"learning" + -0.134*"biology" + -0.123*"assay" + -0.103*"algorithm" + -0.102*"molecu

In [16]:
get_dominant_topics(jobs_listings_tfidf_lsamodel, jobs_listings_tfidf_corpus)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
1202,7.0,0.5425,"laboratory, cell, learning, machine, hospital,...",work shift daywork week variesjob summarythe m...
1210,7.0,0.5419,"laboratory, cell, learning, machine, hospital,...",work shift eveningwork week variesjob summaryt...
1201,7.0,0.5417,"laboratory, cell, learning, machine, hospital,...",work shift nightwork week variesjob summarythe...
1197,7.0,0.5412,"laboratory, cell, learning, machine, hospital,...",work shift thirdwork week variesjob summary082...
1118,7.0,0.5404,"laboratory, cell, learning, machine, hospital,...",work shift daywork week variesjob summarythe m...
1183,7.0,0.5370,"laboratory, cell, learning, machine, hospital,...",work shift daywork week m fjob summary022019j...
1222,7.0,0.5229,"laboratory, cell, learning, machine, hospital,...",work shift nightwork week variesjob summaryjob...
1141,7.0,0.5226,"laboratory, cell, learning, machine, hospital,...",work shift daywork week variesjob summarythe m...
1239,7.0,0.5018,"laboratory, cell, learning, machine, hospital,...",work shift nightwork week m fjob summaryfull ...
1234,7.0,0.4980,"laboratory, cell, learning, machine, hospital,...",work shift daywork week m f w occasional week...


## LDA Model

### Result using LDA model + Bag of words

In [17]:
jobs_listings_bow_ldamodel = gensim.models.LdaMulticore(jobs_listings_bow_corpus, num_topics=10, id2word=jobs_listings_dictionary)

In [18]:
jobs_listings_bow_ldamodel.print_topics(num_words=7)

[(0,
  '0.007*"development" + 0.005*"research" + 0.004*"project" + 0.004*"analysis" + 0.004*"product" + 0.004*"technical" + 0.004*"required"'),
 (1,
  '0.006*"product" + 0.006*"model" + 0.006*"development" + 0.006*"analysis" + 0.005*"process" + 0.005*"learning" + 0.005*"tool"'),
 (2,
  '0.007*"analysis" + 0.006*"analytics" + 0.005*"development" + 0.005*"product" + 0.005*"process" + 0.004*"learning" + 0.004*"project"'),
 (3,
  '0.006*"development" + 0.006*"research" + 0.005*"solution" + 0.005*"analytics" + 0.005*"support" + 0.005*"analysis" + 0.004*"product"'),
 (4,
  '0.007*"development" + 0.006*"analysis" + 0.006*"information" + 0.005*"support" + 0.005*"product" + 0.005*"solution" + 0.005*"engineering"'),
 (5,
  '0.006*"learning" + 0.005*"analysis" + 0.005*"development" + 0.004*"service" + 0.004*"research" + 0.004*"machine" + 0.004*"analytics"'),
 (6,
  '0.007*"product" + 0.006*"development" + 0.006*"analytics" + 0.005*"learning" + 0.005*"analysis" + 0.005*"management" + 0.004*"machin

In [19]:
pyLDAvis.gensim_models.prepare(jobs_listings_bow_ldamodel, jobs_listings_bow_corpus, jobs_listings_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.007760  0.002907       1        1  20.937704
4     -0.005701 -0.003174       2        1  13.859529
1     -0.005167 -0.006115       3        1  12.794331
0      0.001495  0.007140       4        1  11.377007
9     -0.001187 -0.006602       5        1  10.233181
3     -0.001169  0.000248       6        1   8.608881
2     -0.000656  0.003278       7        1   7.875613
8     -0.000429  0.009977       8        1   6.281887
6      0.001174 -0.006019       9        1   5.894675
5      0.019399 -0.001642      10        1   2.137193, topic_info=            Term         Freq        Total Category  logprob  loglift
93      learning  3742.000000  3742.000000  Default  30.0000  30.0000
7       analysis  4702.000000  4702.000000  Default  29.0000  29.0000
139     research  3371.000000  3371.000000  Default  28.0000  28.0000
195    analytics  3685.000000  3685.000000  Default  27.0000  27.0000
45   development  4912.000000  4912.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
43        degree    46.414969  2768.688417  Topic10  -5.9976  -0.2428
494  information    48.423810  3326.543175  Topic10  -5.9552  -0.3840
427     solution    49.125610  3814.300201  Topic10  -5.9409  -0.5065
408      process    47.354333  3274.338274  Topic10  -5.9776  -0.3905
56   engineering    46.612754  3031.257687  Topic10  -5.9934  -0.3292

[845 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
37255      1  0.229718    abilitynice
37255      6  0.459435    abilitynice
0          1  0.184866         access
0          2  0.154431         access
0          3  0.127377         access
...      ...       ...            ...
18062      7  0.598698      工作ead恐被取消
18063      7  0.598810          年起共累计
18065      7  0.598557      毕业生以及正在寻找
18066      7  0.598511     特朗普zf也出台新政
18067      7  0.598395  确认实习企业及职业规划方向

[2248 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 2, 1, 10, 4, 3, 9, 7, 6])

In [20]:
get_dominant_topics(jobs_listings_bow_ldamodel, jobs_listings_bow_corpus)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
2949,6.0,0.9990,"product, development, analytics, learning, ana...",energy industry transformationthe trillion dol...
1191,0.0,0.9986,"development, research, project, analysis, prod...",about usharris health is a nationally recogniz...
1209,0.0,0.9986,"development, research, project, analysis, prod...",about usharris health is a nationally recogniz...
1208,0.0,0.9985,"development, research, project, analysis, prod...",about usharris health is a nationally recogniz...
460,6.0,0.9984,"product, development, analytics, learning, ana...",the department of energy is seeking motivated ...
3265,4.0,0.9984,"development, analysis, information, support, p...",introductionas a data scientist at ibm you wil...
3160,4.0,0.9983,"development, analysis, information, support, p...",introductionas a data scientist at ibm you wil...
2972,4.0,0.9983,"development, analysis, information, support, p...",introductionas a data scientist at ibm you wil...
2505,4.0,0.9983,"development, analysis, information, support, p...",introductionas a data scientist at ibm you wil...
324,1.0,0.9982,"product, model, development, analysis, process...",job descriptiontitle senior data scientistdesc...


### Result using LDA model + TF-IDF

In [21]:
jobs_listings_tfidf_ldamodel = gensim.models.LdaMulticore(jobs_listings_tfidf_corpus, num_topics=10, id2word=jobs_listings_dictionary)

In [22]:
jobs_listings_tfidf_ldamodel.print_topics(num_words=7)

[(0,
  '0.001*"learning" + 0.001*"research" + 0.001*"machine" + 0.001*"model" + 0.001*"client" + 0.001*"customer" + 0.001*"engineer"'),
 (1,
  '0.001*"product" + 0.001*"learning" + 0.001*"analysis" + 0.001*"clinical" + 0.001*"client" + 0.001*"insight" + 0.001*"analytics"'),
 (2,
  '0.001*"bull" + 0.001*"learning" + 0.001*"machine" + 0.001*"analytics" + 0.001*"model" + 0.001*"product" + 0.001*"research"'),
 (3,
  '0.001*"product" + 0.001*"machine" + 0.001*"learning" + 0.001*"analytics" + 0.001*"development" + 0.001*"client" + 0.001*"model"'),
 (4,
  '0.001*"learning" + 0.001*"machine" + 0.001*"analytics" + 0.001*"product" + 0.001*"solution" + 0.001*"model" + 0.001*"research"'),
 (5,
  '0.001*"analytics" + 0.001*"product" + 0.001*"database" + 0.001*"research" + 0.001*"management" + 0.001*"client" + 0.001*"cell"'),
 (6,
  '0.001*"analytics" + 0.001*"learning" + 0.001*"customer" + 0.001*"model" + 0.001*"machine" + 0.001*"solution" + 0.001*"product"'),
 (7,
  '0.001*"learning" + 0.001*"mach

In [23]:
pyLDAvis.gensim_models.prepare(jobs_listings_tfidf_ldamodel, jobs_listings_tfidf_corpus, jobs_listings_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.017763  0.000974       1        1  15.891422
3     -0.005701 -0.005390       2        1  11.343244
6     -0.001478 -0.005352       3        1  10.805424
2      0.003627  0.004940       4        1  10.433494
1      0.005266 -0.009725       5        1   9.525574
7      0.001993  0.000602       6        1   8.972393
4      0.007240  0.002201       7        1   8.646377
8     -0.003179 -0.006427       8        1   8.635581
0     -0.002253  0.017119       9        1   8.184540
9      0.012247  0.001059      10        1   7.561950, topic_info=                Term      Freq      Total Category  logprob  loglift
19841           bull  7.000000   7.000000  Default  30.0000  30.0000
12781          tutor  7.000000   7.000000  Default  29.0000  29.0000
8233         quantum  4.000000   4.000000  Default  28.0000  28.0000
800          trading  8.000000   8.000000  Default  27.0000  27.0000
2699   environmental  8.000000   8.000000  Default  26.0000  26.0000
...              ...       ...        ...      ...      ...      ...
8         analytical  1.260567  18.449868  Topic10  -7.7356  -0.1015
427         solution  1.264100  22.470708  Topic10  -7.7328  -0.2958
408          process  1.242643  20.246081  Topic10  -7.7500  -0.2087
522      requirement  1.234266  18.592334  Topic10  -7.7567  -0.1302
106         modeling  1.231090  18.362105  Topic10  -7.7593  -0.1203

[798 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
191       1  0.144249   able
191       2  0.144249   able
191       3  0.144249   able
191       4  0.072125   able
191       5  0.144249   able
...     ...       ...    ...
310       6  0.164774  youll
310       7  0.082387  youll
310       8  0.082387  youll
310       9  0.082387  youll
310      10  0.082387  youll

[1239 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 7, 3, 2, 8, 5, 9, 1, 10])

In [24]:
get_dominant_topics(jobs_listings_tfidf_ldamodel, jobs_listings_tfidf_corpus)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
1179,2.0,0.9488,"bull, learning, machine, analytics, model, pro...",work shift daywork week m fjob summarywe are ...
289,2.0,0.9482,"bull, learning, machine, analytics, model, pro...",senior data scientist description the senior d...
1014,1.0,0.9479,"product, learning, analysis, clinical, client,...",is the trusted platform that makes data acces...
3261,6.0,0.9473,"analytics, learning, customer, model, machine,...",why join technologies leverages technology to...
1141,6.0,0.9469,"analytics, learning, customer, model, machine,...",work shift daywork week variesjob summarythe m...
3359,6.0,0.9468,"analytics, learning, customer, model, machine,...",we are hiringdata engineer fulltime day aust...
1702,2.0,0.9466,"bull, learning, machine, analytics, model, pro...",loc1300mktwanamaker building req id 57161shift...
1199,6.0,0.9465,"analytics, learning, customer, model, machine,...",work shift eveningwork week m fjob summary082...
314,1.0,0.9465,"product, learning, analysis, clinical, client,...",explore the possibilities across our global ho...
3186,3.0,0.9464,"product, machine, learning, analytics, develop...",location pickle research campusjob posting tit...
